# Python 101 
## Part VII.

---

## Web scraping

### 0. Easy file sharing
Start your own web-server:
- in command prompt change your directory to the notebook directory
- start the server with the `python -m http.server` command

### 1. Obtain a webpage

The easiest way is to use a third party library called __`requests`__. Let's import it right away!

In [ ]:
import requests

And then we simply ask a server to give us an html document by requesting it through an url.

In [ ]:
existing_url = 'http://localhost:8000/test.html'
response = requests.get(existing_url)
print(response.status_code) # hopefully 200 -> successful download

In [ ]:
not_existing_url = 'http://localhost:8000/test1.html'
response = requests.get(not_existing_url)
print(response.status_code) # unfortunately 404 -> not exists

__Common status codes:__
- 200: success
- 301: permanent redirect
- 303: redirect
- 400: bad request
- 401: unauthorized
- 404: not exists
- 500: internal server error

In [ ]:
response = requests.get(existing_url)
print(response.content.decode('utf-8'))

Jupyter can render the page if it was successfully downloaded.

In [ ]:
from IPython.display import HTML
if response.status_code == 200:
    result = HTML(response.content.decode('utf-8'))
else:
    result = 'Nah, let\'s have a beer instead!'

In [ ]:
result

### 2. Process HTML

#### Story time: The skeleton of a html document

__HTML__ is a markup language, its basic build blocks are the `<tag>`s.<br>
(Almost) every `<tag>` has two parts:

- Opening `<tag>` 
- Closing `</tag>` 

Important html `<tag>`s:

- `<html></html>`
- `<head></head>`
- `<body></body>`
- `<h1></h1>`, ..., `<h6></h6>`
- `<div></div>`
- `<p></p>`
- `<span></span>`
- `<section></section>`
- `<a href=""></a>`
- `<img src="">`
- `<br>`
- ```
<table>
    <thead>
        <tr>
            <th></th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td></td>
            ...
        </tr>
    </tbody>
</table>
 ```
- `<ul></ul>` / `<ol></ol>` + `<li></li>`
    
Tags can have different attributes:
- `<a>`: href
- `<img>`: src
- id
- class
- anything that is not a html keyword
    

#### Let's parse it!

We have a third party module for this purpose as well, the __`BeautifulSoup`__.  
Let's import it!

In [ ]:
from bs4 import BeautifulSoup

Then create a soup from the downloaded document.

In [ ]:
document = response.content
soup = BeautifulSoup(document, 'html.parser')

In [ ]:
print(soup.prettify())

With the created soup (which is a parsed document) we can easily access any part of the document.  
Let's try to:
- get the title of the document

In [ ]:
print(soup.title)
print(type(soup.title))

- get the title text

In [ ]:
print(soup.title.get_text())
print(type(soup.title.get_text()))

- get the text-only version of the page

In [ ]:
print(soup.get_text())

- get all the links from the document

In [ ]:
soup.find_all('a')

- get the actual urls from the tags

In [ ]:
for url in soup.find_all('a'):
    print(url.get('href'))

During scraping, there are a lot of different tasks that must be solve in order to get the data we need. 
In this case this demo document has important and unimportant parts. We only need the important parts.   
#### a) Let's find the important links!

In [ ]:
important_urls = []
for url in soup.find_all('a'):
    if 'important_part' in url.get('href'):
        important_urls.append(url.get('href'))
print(important_urls)

#### b) Let's find the important text in the document
- select every paragraph which has "important" class

In [ ]:
soup.find_all('p', {'class': 'important'})
# or:
soup.find_all('p', class_='important')

- Whooops, something's going on! Let's investigate!

In [ ]:
important_paragraphs = soup.find_all('p', class_='important')

- print the text in the tags, and tags' parent's id attribute

In [ ]:
for p in important_paragraphs:
    print(p.get_text(), '>', p.parent.get('id'))

- We can see, that the "fake" result is from somewhere else

In [ ]:
print(soup.find(id='not_main_section'))

- We have a hidden fake section! Let's modify our search!

In [ ]:
soup.find(id='main_content').find_all('p', class_='important')

#### c) Let's find the pictures of our interest
- Let's have the "nice" pictures from the div with random_images_1 class!

In [ ]:
(
    soup
    .find(id='main_content')
    .find('div', class_='random_images_1')
    .find_all('img', class_='nice')
)

- Whoops again. Filter out the result we don't like.

In [ ]:
imgs = (
    soup
    .find(id='main_content')
    .find('div', class_='random_images_1')
    .find_all('img', class_='nice')
)
nice_imgs = []
for img in imgs:
    if 'not' not in img.get('class'):
        nice_imgs.append(img.get('src'))
print(nice_imgs)

Most important methods:
- `.find(tag, id, class_, attrs)`
- `.find_all(tag, id, class_, attrs)`
- `.get(attribute)`
- `.get_text()`

#### Exercise:
- Find every **visible** headlines (`h1`...`h6`) texts and subtitles

---

## Let's do some...

<img align="left" width=150 src="pics/magic.gif">
<br style="clear:left;"/>

### Cool library of the week: tqdm

#### A progressbar to follow the progress of your computation

- import and try it!

In [ ]:
!conda install tqdm -y

In [ ]:
import time
from tqdm import tqdm

In [ ]:
for i in tqdm(range(100)):
    time.sleep(.01)

- Let's use it with the LOTR example

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
import nltk

url ='http://ae-lib.org.ua/texts-c/tolkien__the_lord_of_the_rings_{book}__en.htm'
LOTR = requests.get(url.format(book=1)).content
LOTR = BeautifulSoup(LOTR, "html.parser").getText()

def needed(token):
    stopword = token not in nltk.corpus.stopwords.words('english')
    number = not token.isnumeric()
    length = len(token) > 1 # can be 2 as well
    return stopword and number and length

In [ ]:
tokens = filter(needed, tqdm(nltk.word_tokenize(LOTR.lower())))

wordcount = nltk.FreqDist(tokens)
wordcount.most_common(25)

## It's your turn - write the missing code snippets!

#### 1. Save every important link to a file from the example page

In [ ]:
BASE_URI = './data/'
filename = 'important_urls.txt'
# your code goes here

#### 2. Let's get a random post from bash.hu!
- get the page from http://bash.hu/random
- posts are contained in __`div`__ tags with __`qtxt`__ class
- print the text

In [ ]:
URI = "http://bash.hu/random"
# your code goes here

#### 3. Put the previous code into a function with two arguments: number of posts, and output filename

In [ ]:
def i_want_fun(output, times=5):
    pass # your code goes here

In [ ]:
i_want_fun(BASE_URI+'fun.txt')

#### 4. Create a class from the previous function. 
The class should store all of the post texts.
The class should have a method:
 - called `crawl` which crawls one random bash.hu post
 - called `crawl_multiple` which crawls a number (given as argument) of bash.hu posts
 - called `show_posts` which prints out the crawled posts
 - called `export` which saves the posts into a file (filename is given as argument)
 - called `reset` which empties the posts

I already created the class' skeleton for you. Write your code in place of the `pass` statements.

In [ ]:
class IWantFun(object):
    
    URI = "http://bash.hu/random"
    
    def __init__(self):
        pass
    
    def crawl(self):
        pass
    
    def crawl_multiple(self, times=5):
        pass
    
    def show_urls(self):
        pass
    
    def export(self, output):
        pass
    
    def reset(self):
        pass

In [ ]:
nine = IWantFun()
nine.crawl()
nine.show_urls()
nine.crawl_multiple(5)
nine.show_posts()
nine.export(BASE_URI + 'fun.txt')
nine.reset()
nine.show_urls()